In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, Flatten, BatchNormalization, Lambda

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
def create_dynamic_cnn(input_dim, num_classes, sliding_window_size=3):
    """
    Creates a CNN dynamically based on input dimensions and label classes,
    with a decreasing number of filters in successive layers.
    """
    # Input layer
    input_layer = Input(shape=(input_dim,))
    
    # Reshape input to be compatible with Conv1D using Lambda layer
    reshaped_input = Lambda(lambda x: tf.expand_dims(x, axis=-1))(input_layer)  # Shape: (batch_size, input_dim, 1)
    
    # Determine number of convolutional layers based on input size
    num_conv_layers = max(2, input_dim // 10)  # At least 2 layers
    filters = input_dim * 2  # Start with a reasonable number of filters
    x = reshaped_input

    # Add convolutional layers with decreasing filter counts
    for _ in range(num_conv_layers):
        x = Conv1D(filters=filters, kernel_size=sliding_window_size, activation='relu', padding='same', strides=1)(x)
        x = BatchNormalization()(x)
        x = Dropout(0.2)(x)
        filters = int(max(16, filters // 2))  # Gradually decrease filters, but not below 16
    
    # Flatten the output of the final Conv1D layer
    x = Flatten()(x)
    
    # Fully connected layers
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)
    
    # Create model
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [5]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Initialize the one-hot encoder for the target
    encoder = OneHotEncoder(sparse_output=False)

    print("Reading training data...")
    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        train_data['label'] = train_data['label'].map(label_mapping)
    train_X = train_data.drop(columns=['label']).values
    train_y = train_data['label'].values
    train_y = encoder.fit_transform(train_y.reshape(-1, 1))
    del train_data
    gc.collect()

    print("Reading testing data...")
    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(columns=['label']).values
    test_y = test_data['label'].values
    test_y = encoder.transform(test_y.reshape(-1, 1))
    del test_data
    gc.collect()

    # EarlyStopping Callback (optional, to avoid overfitting)
    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

    # Number of runs for averaging results
    num_runs = 10

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Train the Model with Validation Split N tines for more accurate metrics
    #print("Verbose output only for first run...")
    verbose_run = 0
    for run in range(num_runs):

        print("Creating model...")
        # Model is defined separately in each run, since the random combination layers
        # must be randomly initialized each time. Otherwise, the "random" indices stay the same
        # throughout all runs
        model = create_dynamic_cnn(train_X.shape[1], test_y.shape[1])
        print("Model created.")

        print(f"Run {run + 1}/{num_runs} started...")
        history = model.fit(
            train_X, train_y, 
            epochs=100, 
            batch_size=int(train_X.shape[0] * 0.01),
            callbacks=[early_stopping],
            verbose=verbose_run
        )
        verbose_run = 0 # Suppress detailed output for multiple runs
        print("Run finished. Evaluating metrics...")

        test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
        y_pred = model.predict(test_X, verbose=0)
        y_pred_classes = y_pred.argmax(axis=1)
        y_true_classes = test_y.argmax(axis=1)
        del model
        gc.collect()

        # Compute metrics
        balanced_acc = balanced_accuracy_score(y_true_classes, y_pred_classes)
        roc_auc = roc_auc_score(test_y, y_pred, multi_class='ovr')  # `test_y` is fine here for AUC
        pr_auc = average_precision_score(test_y, y_pred, average='weighted')
        f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
        mcc = matthews_corrcoef(y_true_classes, y_pred_classes)

        # Store metrics
        metrics_storage['test_loss'].append(test_loss)
        metrics_storage['test_accuracy'].append(test_acc)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

        # Average the metrics over all successful runs
        print(f"\nAggregated Metrics for {run+1} runs:")
        for metric, values in metrics_storage.items():
            avg_value = np.mean(values)
            print(f"{metric}: {avg_value:.4f}")

    gc.collect()
        

In [6]:
run_model("/kaggle/input/ma-datasets/shuttle_train.csv", "/kaggle/input/ma-datasets/shuttle_test.csv", is_string_labels = False)

Reading training data...
Reading testing data...
Creating model...
Model created.
Run 1/10 started...
Run finished. Evaluating metrics...

Aggregated Metrics for 1 runs:
test_loss: 0.0020
test_accuracy: 0.9991
balanced_accuracy: 0.8762
roc_auc: 1.0000
pr_auc: 1.0000
f2: 0.9991
mcc: 0.9976
0_precision: 0.9993
0_recall: 0.9998
0_f1-score: 0.9996
0_support: 9117.0000
1_precision: 0.9091
1_recall: 1.0000
1_f1-score: 0.9524
1_support: 10.0000
2_precision: 0.9429
2_recall: 0.9706
2_f1-score: 0.9565
2_support: 34.0000
3_precision: 1.0000
3_recall: 0.9978
3_f1-score: 0.9989
3_support: 1781.0000
4_precision: 0.9985
4_recall: 0.9985
4_f1-score: 0.9985
4_support: 653.0000
5_precision: 1.0000
5_recall: 0.5000
5_f1-score: 0.6667
5_support: 2.0000
6_precision: 1.0000
6_recall: 0.6667
6_f1-score: 0.8000
6_support: 3.0000
accuracy: 0.9991
macro avg_precision: 0.9785
macro avg_recall: 0.8762
macro avg_f1-score: 0.9104
macro avg_support: 11600.0000
weighted avg_precision: 0.9992
weighted avg_recall: 0.9

In [ ]:
run_model("/kaggle/input/ma-datasets/covtype_train.csv", "/kaggle/input/ma-datasets/covtype_test.csv", is_string_labels = False)

Run 1/5 started...

Aggregated Metrics for 1 runs:
test_loss: 0.3293
test_accuracy: 0.8658
balanced_accuracy: 0.7690
roc_auc: 0.9854
pr_auc: 0.9405
f2: 0.8650
mcc: 0.7838
0_precision: 0.8909
0_recall: 0.8300
0_f1-score: 0.8594
0_support: 42368.0000
1_precision: 0.8633
1_recall: 0.9154
1_f1-score: 0.8886
1_support: 56661.0000
2_precision: 0.8184
2_recall: 0.8779
2_f1-score: 0.8471
2_support: 7151.0000
3_precision: 0.7817
3_recall: 0.6849
3_f1-score: 0.7301
3_support: 549.0000
4_precision: 0.7846
4_recall: 0.5350
4_f1-score: 0.6362
4_support: 1899.0000
5_precision: 0.7343
5_recall: 0.6461
5_f1-score: 0.6874
5_support: 3473.0000
6_precision: 0.8827
6_recall: 0.8935
6_f1-score: 0.8881
6_support: 4102.0000
accuracy: 0.8658
macro avg_precision: 0.8223
macro avg_recall: 0.7690
macro avg_f1-score: 0.7910
macro avg_support: 116203.0000
weighted avg_precision: 0.8658
weighted avg_recall: 0.8658
weighted avg_f1-score: 0.8645
weighted avg_support: 116203.0000
Run 2/5 started...

Aggregated Metrics

In [ ]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("/kaggle/input/ma-datasets/darknet_train.csv", "/kaggle/input/ma-datasets/darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/5 started...

Aggregated Metrics for 1 runs:
test_loss: 0.0060
test_accuracy: 0.9983
balanced_accuracy: 0.9708
roc_auc: 0.9999
pr_auc: 0.9998
f2: 0.9983
mcc: 0.9937
0_precision: 0.9997
0_recall: 0.9997
0_f1-score: 0.9997
0_support: 26862.0000
1_precision: 0.9981
1_recall: 0.9989
1_f1-score: 0.9985
1_support: 2657.0000
2_precision: 0.9956
2_recall: 0.9923
2_f1-score: 0.9939
2_support: 908.0000
3_precision: 0.9922
3_recall: 0.9713
3_f1-score: 0.9816
3_support: 522.0000
4_precision: 0.9829
4_recall: 0.9829
4_f1-score: 0.9829
4_support: 293.0000
5_precision: 0.9631
5_recall: 0.9703
5_f1-score: 0.9667
5_support: 269.0000
6_precision: 0.9055
6_recall: 0.9914
6_f1-score: 0.9465
6_support: 116.0000
7_precision: 0.8800
7_recall: 0.8302
7_f1-score: 0.8544
7_support: 53.0000
8_precision: 1.0000
8_recall: 1.0000
8_f1-score: 1.0000
8_support: 44.0000
accuracy: 0.9983
macro avg_precision: 0.9686
macro avg_recall: 0.9708
macro avg_f1-score: 0.9694
macro avg_support: 31724.0000
weighted avg_prec